# Animal Classification

## Import the modules

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.layers import Dense,Conv2D,GlobalAvgPool2D,Input
from tensorflow.keras import callbacks,optimizers
import tensorflow as tf

from google.colab import drive
import numpy as np
import os

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/dataset/animals10/

/content/drive/MyDrive/dataset/animals10


In [4]:
!pwd

/content/drive/MyDrive/dataset/animals10


## Download the datasets

In [5]:
from google.colab import files

In [6]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"cmglonly","key":"4cbe382e220eca344f52b23e3eab47a3"}'}

In [8]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d alessiocorrado99/animals10

100% 584M/586M [00:03<00:00, 179MB/s]
100% 586M/586M [00:03<00:00, 174MB/s]


In [10]:
!ls

animals10.zip  kaggle.json


In [11]:
!unzip animals10.zip

Streaming output truncated to the last 5000 lines.
  inflating: raw-img/ragno/OIP-MVQChNQpHiMZEVMPehrvoAHaE6.jpeg  
  inflating: raw-img/ragno/OIP-MVdIFDAFpNyK_XCpwkDIWgHaFj.jpeg  
  inflating: raw-img/ragno/OIP-MYY_8k5AHqMJwTCNXfH6rwHaFj.jpeg  
  inflating: raw-img/ragno/OIP-MZEVXuoMsATG8u_UuTQ6hAHaE8.jpeg  
  inflating: raw-img/ragno/OIP-MZg0tznDRE95AMfyuutuYAHaE6.jpeg  
  inflating: raw-img/ragno/OIP-M_r9DkR2X5gP0P5-z8wPYQHaE7.jpeg  
  inflating: raw-img/ragno/OIP-M_xDUSzjlF3-w9XHCm93cgHaFF.jpeg  
  inflating: raw-img/ragno/OIP-MbI4-bMyutRX3X382IAVcQAAAA.jpeg  
  inflating: raw-img/ragno/OIP-MbMksZdD7gomsC9fd6cpKgHaHJ.jpeg  
  inflating: raw-img/ragno/OIP-Md4AMcvjaHcivJqeW06wEwHaFW.jpeg  
  inflating: raw-img/ragno/OIP-MgCgailkT5XkO1XczzQCngHaLK.jpeg  
  inflating: raw-img/ragno/OIP-MgrKq05YHGXsgj1AmYIyJQHaE8.jpeg  
  inflating: raw-img/ragno/OIP-MhRGx7d8nbnHBuqrsO00EQHaFo.jpeg  
  inflating: raw-img/ragno/OIP-MiVgMXcKGQdog2KGe7i0YQHaHh.jpeg  
  inflating: raw-img/ragno/OIP-Mis0lJf3

## Fixing the directory name

In [12]:
for i in os.listdir('raw-img'):
    print(i)

cane
cavallo
elefante
farfalla
gallina
gatto
mucca
pecora
ragno
scoiattolo


In [13]:
from translate import translate

In [14]:
translate

{'butterfly': 'farfalla',
 'cane': 'dog',
 'cat': 'gatto',
 'cavallo': 'horse',
 'chicken': 'gallina',
 'cow': 'mucca',
 'dog': 'cane',
 'elefante': 'elephant',
 'elephant': 'elefante',
 'farfalla': 'butterfly',
 'gallina': 'chicken',
 'gatto': 'cat',
 'mucca': 'cow',
 'pecora': 'sheep',
 'scoiattolo': 'squirrel',
 'spider': 'ragno',
 'squirrel': 'scoiattolo'}

In [15]:
os.listdir('raw-img')

['cane',
 'cavallo',
 'elefante',
 'farfalla',
 'gallina',
 'gatto',
 'mucca',
 'pecora',
 'ragno',
 'scoiattolo']

In [16]:
# This two block below is used because there are wrong pair in the 'translate' dictionary
os.rename("raw-img/ragno", "raw-img/spider")
os.rename("raw-img/scoiattolo", "raw-img/squirell")    

In [17]:
for i in os.listdir('raw-img'):
    try:
        os.rename('raw-img/'+i, 'raw-img/'+translate[i])
    except Exception as e:
        print(e)

'squirell'


In [18]:
!ls raw-img

butterfly  cat	chicken  cow  dog  elephant  horse  ragno  sheep  squirell


In [19]:
for i in os.listdir('raw-img'):
  print(i + ':', len(os.listdir('raw-img/'+i)))

dog: 4863
horse: 2623
elephant: 1446
butterfly: 2112
chicken: 3098
cat: 1668
cow: 1866
sheep: 1820
ragno: 4821
squirell: 1862


As we can see, that the data is unbalanced, so let remove unecessary data and save it into train and test directory

In [20]:
try:
  os.mkdir('train')
  os.mkdir('test')
except:
  pass

for i in os.listdir('raw-img'):
  try:
    os.mkdir('train/'+i)
    os.mkdir('test/'+i)
  except:
    pass
  for j in os.listdir('raw-img/'+i)[:1000]:
    os.rename('raw-img/'+i+'/'+j, 'train/'+i+'/'+j)
  for j in os.listdir('raw-img/'+i)[:400]:
    os.rename('raw-img/'+i+'/'+j, 'test/'+i+'/'+j)

In [21]:
!ls train

butterfly  cat	chicken  cow  dog  elephant  horse  ragno  sheep  squirell


In [22]:
!ls test

butterfly  cat	chicken  cow  dog  elephant  horse  ragno  sheep  squirell


In [23]:
def img_data(dir_path, target_size, batch, class_lst, preprocessing):
  if preprocessing:
    gen_object = ImageDataGenerator(preprocessing_function=preprocessing)
  else:
    gen_object = ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path, 
                                   target_size=target_size, 
                                   batch_size=batch, 
                                   class_mode='sparse', 
                                   classes=class_lst,
                                   shuffle=True))

In [24]:
train_data_generator = img_data('train', (224,224), 500, os.listdir('train'), preprocess_input)
valid_data_generator = img_data('test', (224,224), 500, os.listdir('test'), preprocess_input)

Found 10000 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.


In [25]:
train_data_generator

In [26]:
base_model = tf.keras.applications.MobileNetV2(
             input_shape=(224,224,3), alpha=1.0, include_top=False, weights='imagenet',
             input_tensor=None, pooling=None, classes=1000,
             classifier_activation='softmax'            
             )

9412608/9406464 [==============================] - 0s 0us/step


In [27]:
base_model.trainable = False

In [28]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [29]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1311744   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total params: 3,579,978
Trainable params: 1,321,994
Non-trainable params: 2,257,984
_________________________________________________________________


In [31]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
early_stop = callbacks.EarlyStopping(monitor='loss', patience=5, mode='min')
save_checkpoint = callbacks.ModelCheckpoint('.animal_model.hdf5', save_best_only=True, monitor='loss', mode='min')

In [34]:
model.fit(train_data_generator, batch_size=500, validation_data=valid_data_generator, callbacks=[early_stop, save_checkpoint], epochs=10)

Epoch 1/10
20/20 [==============================] - 80s 2s/step - loss: 0.4111 - accuracy: 0.8748 - val_loss: 0.2219 - val_accuracy: 0.9383


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/10
20/20 [==============================] - 44s 2s/step - loss: 0.1159 - accuracy: 0.9653 - val_loss: 0.1745 - val_accuracy: 0.9523
Epoch 3/10
20/20 [==============================] - 44s 2s/step - loss: 0.0755 - accuracy: 0.9802 - val_loss: 0.1709 - val_accuracy: 0.9520
Epoch 4/10
20/20 [==============================] - 44s 2s/step - loss: 0.0506 - accuracy: 0.9879 - val_loss: 0.1986 - val_accuracy: 0.9477
Epoch 5/10
20/20 [==============================] - 44s 2s/step - loss: 0.0338 - accuracy: 0.9934 - val_loss: 0.1890 - val_accuracy: 0.9485
Epoch 6/10
20/20 [==============================] - 44s 2s/step - loss: 0.0223 - accuracy: 0.9969 - val_loss: 0.1841 - val_accuracy: 0.9528
Epoch 7/10
20/20 [==============================] - 44s 2s/step - loss: 0.0178 - accuracy: 0.9977 - val_loss: 0.1876 - val_accuracy: 0.9528
Epoch 8/10
20/20 [==============================] - 44s 2s/step - loss: 0.0130 - accuracy: 0.9988 - val_loss: 0.1904 - val_accuracy: 0.9503
Epoch 9/10
20/20 [==

## Let's get into the deployment

*Don't forget to download the hdf5 file*

In [36]:
model.save('animal_model.hdf5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
